The `JsonWriter` is the type class for serialising to JSON:

In [1]:
object JsonModel {
  sealed trait MyJson
  final case class JsObject(get: Map[String, MyJson]) extends MyJson
  final case class JsString(get: String) extends MyJson
  final case class JsNumber(get: Double) extends MyJson
  final case object JsNull extends MyJson

  //  type class
  trait JsonWriter[A] {
    def write(value: A): MyJson
  }
}

defined object JsonModel

In [2]:
// Example model to demonstrate
object PersonModel {
    final case class Person(name: String, email: String)
}

defined object PersonModel

## Type class instances
implement a concrete class.

In Scala, we define instances by creating concrete implementations of the type class and tagging them with the implicit keyword:


In [3]:
import JsonModel._
import PersonModel._

// implicit values: type class instances
object JsonWriterInstances {
    implicit val stringWriter: JsonWriter[String] = new JsonWriter[String] {
        def write(value: String): MyJson = JsString(value)
    }

    implicit val personWriter: JsonWriter[Person] = new JsonWriter[Person] {
        def write(value: Person): MyJson = JsObject(
            Map("name" -> JsString(value.name)
                ,"email" -> JsString(value.email)))
    }


    // ... implement others to return such as JsNumber 
}

import JsonModel._
import PersonModel._
defined object JsonWriterInstances

Above are known as implicit values.

Type classes in Scala are implemented using implicit values and parameters, and optionally using implicit classes. Scala language constructs correspond to the components of type classes as follows:

- traits: type classes;
- implicit values: type class instances;
- implicit parameters: type class use; and
- implicit classes: optional utilities that make type classes easier to use.

In the cats:
- interface objects
- interface syntax

## interface objects:

In [4]:
// companion object
object MyJson {
    def toJson[A](value: A)(implicit w: JsonWriter[A]): MyJson = w.write(value)
}

defined object MyJson

For example to use the above:

In [5]:
import JsonWriterInstances._

MyJson.toJson(Person("Ojitha", "ojitha@test.com")) //implicit scope applied

import JsonWriterInstances._
res5_1: MyJson = JsObject(
  get = Map(
    "name" -> JsString(get = "Ojitha"),
    "email" -> JsString(get = "ojitha@test.com")
  )
)

## Interface Syntax
We can alternatively use extension methods to extend existing types with interface methods

In [6]:
object JsonSyntax {
    implicit class JsonWriterOps[A](value: A) {
        def convert2Json(implicit w: JsonWriter[A]): MyJson = w.write(value)
    }
}

defined object JsonSyntax

Use the interface Syntax:

In [7]:
import JsonWriterInstances._
import JsonSyntax._

Person("Ojitha", "ojitha@test.com").convert2Json 

import JsonWriterInstances._
import JsonSyntax._
res7_2: MyJson = JsObject(
  get = Map(
    "name" -> JsString(get = "Ojitha"),
    "email" -> JsString(get = "ojitha@test.com")
  )
)

> Working with type classes in Scala means working with implicit values and implicit parameters. Placing instances in a companion object to the type class has special significance in Scala because it plays into something called **implicit scope**.

## Implicit Scope
The implicit scope which roughly consists of:

- local or inherited definitions
- imported definitions
- definitions in the companion object of the type class or the parameter type (in this case `JsonWriter` or `String`).

Definitions are only included in implicit scope if they are tagged with the `implicit` keyword.

Type class instances can be placed roughly four ways:

1. In an object (eg:`JasonWriterInstances`): need to import
2. In a trait: using inheritance
3. In the companion object of the Type Class
4. In the companion object of the parameter type

Type class instances can be defined in two ways:
1. concrete instances as `implicit val`
2. by defining `implicit` methods to construct instances from other type class instances

Above second way created hierarcy:

```scala
MyJson.toJson(Option("A string"))(optonWriter[String]) // apply first
                                         ⬇️
MyJson.toJson(Option("A string"))(optonWriter(stringWriter)) // apply then
```

In [8]:
implicit def optionWriter[A](implicit writer: JsonWriter[A]): JsonWriter[Option[A]] =
    new JsonWriter[Option[A]] {
        def write(option: Option[A]): MyJson = option match {
            case Some(value) => writer.write(value)
            case None => JsNull
        }
    }

defined function optionWriter

In [9]:
MyJson.toJson(Option("A string"))

res9: MyJson = JsString(get = "A string")

> `implicit` methods with non‐implicit parameters form a different Scala pattern called an **implicit conversion**, which is an old programming concept.